In [1]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import xgboost
import pandas as pd
import numpy as np
import pickle
from sklearn.preprocessing import LabelEncoder
from pathlib import Path
from tqdm.auto import tqdm

tqdm.pandas()

In [2]:

df = pd.read_parquet('C:/Saravana/Projects/Intellizenz/intellizenz-model-training/data/export_features_2016_2020_v2.parquet.gzip')

In [3]:
pd.set_option('display.max_columns', None)

In [4]:
df=df.rename(columns = {'band_x':'band', 'vg_state_x':'vg_state'})

In [5]:
display(df.head(5))

,imp_id,gj,import,mufo_referenz_n,barcode_nr,veranst_segment,rekla_jn,vg_datum_von,vg_ort,vg_raum,nutzfall,nutzfall_raum,musikleiter_name,kapelle_name,tarif_nr,tarif_bez,nutzfall_nr,vg_inkasso,inkasso_netto,inkasso_brutto,veranst_geschaeftszeichen,veranst_name,veranst_strasse,veranst_plz,veranst_ort,nutzliznehm_geschaeftszeichen,nutzliznehm_name,nutzliznehm_vorname,nutzliznehm_strasse,nutzliznehm_ort,location,band,promoter,vg_state,vg_raum_clean,vg_raum_wo_stopwords,vg_raum_keywords_x,vg_datum_year,vg_datum_month,vg_datum_day_of_week,vg_datum_season,vg_raum_keywords_y,place_kirche,place_hotel,place_cafe,place_theater,place_club,place_halle,place_gaststaette,place_festhalle,place_kulturzentrum,place_festzelt,place_schloss,place_pub,place_stadthalle,place_park,place_gasthof,place_kabarett,place_arena,place_schlachthof,place_wandelhalle,place_turnhalle,place_buergerhaus,place_museum,place_rathaus,place_staatsbad,place_zelt,place_jazz,place_forum,place_gymnasium,place_schule,place_sporthalle,vg_state_y,state_bavaria,state_thuringia,state_baden-wuerttemberg,state_rhineland-palatinate,state_brandenburg,state_berlin,state_lower saxony,state_schleswig-holstein,state_hamburg,state_hesse,state_saarland,state_north rhine-westphalia,state_saxony-anhalt,state_saxony,state_mecklenburg-western pomerania,state_bremen,band_y,band_kurorchester bad wil,band_musikverein harmonie,band_kasalla,band_cat ballou,band_roncalli royal orch,band_jugendblasorchester,band_kurorchester bad pyr,band_hoehner,band_paveier,band_domstuermer,band_kluengelkoepp,band_alleinunterhalter,band_the gregorian voices,band_brings,band_musica hungarica,band_concerto,band_bad salzuflen orches,band_musikverein stadtkap,band_salonorchester hunga,band_miljoe,band_raeuber,band_kabarett leipziger f,band_marita koellner,band_salon-orchester hung,band_blaeck foeoess,band_schuelerinnen und sc,band_romain vicente,band_staatliche kurkapell,band_musikzug der freiwil,band_funky marys
ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
9938766,24643,2016,ABRE U INKA 2016,None,5027961235,4,NONRekla,2014-02-08,LISBERG,TURNHALLE SV RW LISBERG,20140208.0,TURNHALLE SV RW LISBERG,STAERK REINHOLD,DAS BOEHMISCHE FEUER,892,U-K (MUSIKER),18293091,210.00,NaN,NaN,1035530900,AURACHTALER BLASMUSIKVEREIN WALSDORF E.V,SCHINDHOLZWEG 17,96194,WALSDORF,1510423300,BAYERISCHER BLASMUSIKVERBAND,None,None,MUENCHEN,LISBERG TURNHALLE SV RW LISBERG,DAS BOEHMISCHE FEUER,AURACHTALER BLASMUSIKVEREIN WALSDORF E.V 96194,BAVARIA,TURNHALLE SV RW LISBERG,turnhalle sv rw lisberg,"[turnhalle, sv, rw, lisberg]",2014,2,5,1,"[turnhalle, sv, rw, lisberg]",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,BAVARIA,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,DAS BOEHMISCHE FEUER,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9939272,24643,2016,ABRE U INKA 2016,None,5053180236,3,NONRekla,2014-02-22,WIPPERDORF,STAATLICHE GRUNDSCHULE WIPPERDORF,20140222.0,STAATLICHE GRUNDSCHULE WIPPERDORF,PATSCHKE RAINER,KYFFHAEUSER TANZ BAN,1087,U-V II. 1 (+II 2),17923031,85.06,NaN,NaN,1374109400,WIPPERDORFER CARNEVAL CLUB E.V,STRASSE DER EINHEIT 6,99752,WIPPERDORF,1374109400,WIPPERDORFER CARNEVAL CLUB,None,STRASSE DER EINHEIT 6,WIPPERDORF,WIPPERDORF STAATLICHE GRUNDSCHULE WIPPERDORF,KYFFHAEUSER TANZ BAN,WIPPERDORFER CARNEVAL CLUB E.V 99752,THURINGIA,STAATLICHE GRUNDSCHULE WIPPERDORF,staatliche grundschule wipperdorf,"[grundschule, wipperdorf]",2014,2,5,1,"[grundschule, wipperdorf]",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,THURINGIA,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,KYFFHAEUSER TANZ BAN,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9939279,24643,2016,ABRE U INKA 2016,None,5053345001,3,NONRekla,2014-02-18,DINGOLFING,BMW-WERK DINGOLFING,20140218.0,BMW-WERK DINGOLFING,JUNDMAYER,BERUFSFACHSCHULE,1087,U-V II. 1 (+II 2),17757433,90.20,NaN,NaN,2000026695,BAYERISCHE MOTORENWERKE AG,MENGKOFENER STR. 13,84130,DINGOLFING,2000026695,BAYERISCHE MOTORE

In [6]:
print(len(df.band.unique()))#317011 - unique bands
print(len(df.promoter.unique()))#176456 - unique promoter/organizer
print(len(df.vg_state.unique()))#16 - unique bundesland/states

317011
176456
16


In [8]:
data_path_inkasso_stats_for_features =Path('C:/Saravana/Projects/Intellizenz/intellizenz-model-training/data/export_inkasso_stats_for_features_2016_2020.parquet.gzip')

data_path_vg_state = Path('C:/Saravana/Projects/Intellizenz/intellizenz-model-training/data/export_vg_stats_for_range_of_vginkasso_2016_2020.parquet.gzip')
data_path_band = Path('C:/Saravana/Projects/Intellizenz/intellizenz-model-training/data/export_band_stats_for_range_of_vginkasso_2016_2020.parquet.gzip')
data_path_promoter = Path('C:/Saravana/Projects/Intellizenz/intellizenz-model-training/data/export_promotor_stats_for_range_of_vginkasso_2016_2020.parquet.gzip')

# stat_var = ['location', 'band_x', 'promoter']
# stat_var = ['band']
stat_var = ['vg_state']
# stat_var = ['promoter']
# stat_var = ['band']
df_stat = {}

def get_descr_stat(row):
    # row represents each line(row) in the dataframe
    def descr_stat(row, var):
        # Initialize a empty pandas dataframe of type float64
        result = pd.Series(dtype='float64')

        if not pd.isnull(row[var]):
            # obtain the list of inkasso values for a specified 'var' attribute like location, band, promoter
            inkasso = df_stat[var].loc[row[var]].copy()
            if len(inkasso) > 1:
                inkasso.remove(row['vg_inkasso'])
                # Returns the mean, count, std, max, min and 0.05, 0.10..0.95 percentiles of the list of vg_inkasso values 
                # for a specific attribute like location, band, promotor
                result = pd.Series(inkasso).describe(percentiles=percentiles)
                result = result.add_prefix('{}_'.format(var))

        result.name = row.name
        return result

    descr_stat_result = pd.Series(dtype='float64')
    for v in stat_var:
        # descr_stat_result = descr_stat_result.append(descr_stat(row, v))
        descr_stat_result = pd.concat([descr_stat_result, descr_stat(row, v)])

    return descr_stat_result


if data_path_inkasso_stats_for_features.exists() :
    df_featurize_descr_stat = pd.read_parquet(data_path_inkasso_stats_for_features)
    
elif data_path_vg_state.exists() and data_path_band.exists() and data_path_promoter.exists():
    df_state = pd.read_parquet(data_path_vg_state)
    df_band = pd.read_parquet(data_path_band)
    df_promoter = pd.read_parquet(data_path_promoter)

    df_featurize_descr_stat = df_state.merge(df_band, how='outer', on='ID').merge(df_promoter,
                                                                                how='outer',
                                                                                on='ID')
else:

    for v in tqdm(stat_var):
        #Group vg_inkasso collected for each location/state, each band and each promoter
        df_stat[v] = df.groupby(v)['vg_inkasso'].apply(list)

    percentiles = [round(x, 2) for x in np.linspace(0, 1, 21)[1:-1].tolist()]

    # Each row of the dataframe 'df' is passed as row argument for get_descr_stat function
    df_featurize_descr_stat = df.progress_apply(get_descr_stat, axis=1)
    # df_featurize_descr_stat.to_pickle('./data/export_descr_stat_2016_2020.pkl.bz2', protocol=4)

display(df_featurize_descr_stat.head(5))

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1742241 [00:00<?, ?it/s]

In [ ]:
df_featurize_descr_stat.to_parquet('C:/Saravana/Projects/Intellizenz/intellizenz-model-training/data/export_band_stats_for_range_of_vginkasso_2016_2020.parquet.gzip',compression='gzip')

In [ ]:
df_featurize_descr_stat.to_parquet('C:/Saravana/Projects/Intellizenz/intellizenz-model-training/data/export_promotor_stats_for_range_of_vginkasso_2016_2020.parquet.gzip',compression='gzip')

In [ ]:
df_featurize_descr_stat.to_parquet('C:/Saravana/Projects/Intellizenz/intellizenz-model-training/data/export_vg_stats_for_range_of_vginkasso_2016_2020.parquet.gzip',compression='gzip')

In [ ]:
# When the time varied vg_inkasso distribution are extracted for vg_state, band, promoter, merge these features
if data_path_vg_state.exists() and data_path_band.exists() and data_path_promoter.exists():
    df_state = pd.read_parquet(data_path_vg_state)
    df_band = pd.read_parquet(data_path_band)
    df_promoter = pd.read_parquet(data_path_promoter)

    df_featurize_descr_stat = df_state.merge(df_band, how='outer', on='ID').merge(df_promoter,
                                                                                how='outer',
                                                                                on='ID')
if not data_path_inkasso_stats_for_features.exists() :
    df_featurize_descr_stat.to_parquet('C:/Saravana/Projects/Intellizenz/intellizenz-model-training/data/export_inkasso_stats_for_features_2016_2020.parquet.gzip',compression='gzip')